In [2]:
from hail import *

In [3]:
import requests

In [4]:
hc = HailContext()

In [5]:
vds=hc.import_vcf('/Users/jma7/Development/server/ga4gh-example-data/release/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf')

In [8]:
vds=vds.split_multi()

In [11]:
vds.write('/Users/jma7/Development/hail/test/chr22.vds')

In [15]:
from guppy import hpy

ImportError: No module named 'guppy'

In [9]:
vds.count()

{'nGenotypes': 2777857472, 'nSamples': 2504, 'nVariants': 1109368}

In [12]:
vds.export_variants("/Users/jma7/Development/hail/test/variantChr22.tsv",'v,v.contig,v.start,v.ref,v.alt')

In [22]:
originalIDs=[]
with open("/Users/jma7/Development/hail/test/variantChr22.tsv","r") as infile:
    for line in infile:
        cols=line.rstrip().split("\t")
        originalIDs.append([cols[0],cols[1],cols[2],cols[3],cols[4]])

In [31]:
resVariants=requests.post("http://192.168.99.105/vatvs/variants/variantsID",json={"ids":originalIDs})

In [34]:
import myvariant

ImportError: No module named 'myvariant'

In [26]:
with open("/Users/jma7/Development/hail/test/hgvsIDs.tsv","w") as outfile:
    outfile.write("VariantID\thgvsID\n")
    for id in resVariants.json():
        outfile.write(id+"\n")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
vdsanno=vds.annotate_variants_table("/Users/jma7/Development/hail/test/hgvsIDs.tsv",root="va.vID",variant_expr="VariantID",config=TextTableConfig(impute=True))

In [ ]:
queryByID=vdsanno.filter_variants_expr('va.vID.hgvsID=="chr22:g.16050115G>A"')

In [ ]:
queryBySample=vdsanno.filter_samples_expr('s.id=="HG00096"')

In [ ]:
queryByID.export_genotypes("/Users/jma7/Development/hail/test/queryByID.tsv",'s,v,g.gt')

In [ ]:
queryBySample.export_genotypes("/Users/jma7/Development/hail/test/queryBySample.tsv",'s,v')